In [1]:
import numpy as np
import pandas as pd

In [ ]:
import keras

Using TensorFlow backend.


ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

SystemError: <class '_frozen_importlib._ModuleLockManager'> returned a result with an error set

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

In [ ]:
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

SystemError: <class '_frozen_importlib._ModuleLockManager'> returned a result with an error set

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Dropout, SpatialDropout1D
from keras.layers import LSTM, Conv1D, MaxPooling1D
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

SystemError: <class '_frozen_importlib._ModuleLockManager'> returned a result with an error set

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

In [2]:
print('reading CSV...')

csv = pd.read_csv('', encoding = "ISO-8859-1", header=None)

reading CSV...


In [3]:
print('parsing CSV...')

X, Y = [], []

for index, row in csv.iterrows():
    X.append(row[5])
    y_part = row[0]
    if y_part == 0:
        yy = np.array([0])
    elif y_part == 4:
        yy = np.array([1])
    else:
        raise Exception('Invalid y_part value=' + y_part)
    Y.append(yy)

parsing CSV...


In [4]:
print('build words map')

max_features = 50000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X)
X, Xt, Y, Yt = train_test_split(X, Y, test_size = 0.3, random_state = random_seed)

validation_size = 1500
X_validate = Xt[-validation_size:]
Y_validate = Yt[-validation_size:]
Xt = Xt[:-validation_size]
Yt = Yt[:-validation_size]

maxlen = 0
def wrap_array(x, maxlen):
    for index in range(len(x)):
        xx = x[index]
        if len(xx) > maxlen:
            maxlen = len(xx)
        x[index] = np.array(xx)
    return np.array(x), maxlen

X, maxlen = wrap_array(X, maxlen)
Xt, maxlen = wrap_array(Xt, maxlen)
X_validate, maxlen = wrap_array(X_validate, maxlen)
Y, maxlen = wrap_array(Y, maxlen)
Yt, maxlen = wrap_array(Yt, maxlen)
Y_validate, maxlen = wrap_array(Y_validate, maxlen)

build words map


In [5]:
print(maxlen)

118


In [ ]:
print('build model')

batch_size = 256

model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(124, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])
print(model.summary())

print('Train...')
model.fit(X, Y, batch_size=batch_size, epochs=2, validation_data=(Xt, Yt), verbose=2)


score, acc = model.evaluate(X_validate, Y_validate, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

build model
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 118, 128)          6400000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 118, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 118, 32)           12320     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 59, 32)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 124)               77872     
_________________________________________________________________
dense_1 (

In [ ]:
import pickle
with open('train_model_large_app','wb') as f:
    pickle.dump(model,f)

In [7]:
from sklearn.externals import joblib 
model = joblib.load('new_train_model')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [9]:
daata = pd.read_csv('/home/tcs/VK_Python_files/Large_app/Restaurant_Reviews.tsv', sep='\t')

In [10]:
daata.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [13]:
final_data = daata.drop(columns=['Liked'])

In [14]:
final_data.head()

,Review
0,Wow... Loved this place.
1,Crust is not good.
2,Not tasty and the texture was just nasty.
3,Stopped by during the late May bank holiday of...
4,The selection on the menu was great and so wer...


In [18]:
sent_list = []
flag = 0
for i in range(len(final_data)):
    #print(final_data['Review'][i])
    tokens = tokenizer.texts_to_sequences([final_data['Review'][i]])
    tokens = pad_sequences(tokens, maxlen=maxlen)
    sentiment = model.predict(np.array(tokens), batch_size=1, verbose = 2)[0][0]
    if (round(sentiment) == 0):
        #print('Negative')
        flag = 0
    else:
        #print('Positive')
        flag = 1
    sent_list.append(flag)
    

In [20]:
final_data['Sentiment'] = sent_list

In [21]:
final_data

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
5,Now I am getting angry and I want my damn pho.,0
6,Honeslty it didn't taste THAT fresh.),0
7,The potatoes were like rubber and you could te...,1
8,The fries were great too.,1
9,A great touch.,1


In [ ]:
print('Trying to predict:')
#text = "This is a presidential decree that Rada's defence committee approved and suggested MPs to support. As some MPs write, there is no agreement on a restriction of certain freedoms(frdm of assembly among them). They also want it written down that elections will take place on March 31"
#text = """The sheer ineptitude of the movie is supposed to be funny, but there's no lunacy behind it: Shore and his writers are like comedians on Prozac, smiling through the fart jokes without a hint of desperation"""
text = 'this is not good'
print(text)
tokens = tokenizer.texts_to_sequences([text])
tokens = pad_sequences(tokens, maxlen=maxlen)
sentiment = model.predict(np.array(tokens), batch_size=1, verbose = 2)[0][0]
print(sentiment)
print('Sentiment =', sentiment*100)

if (round(sentiment) == 0):
    print('Negative')
else:
    print('Positive')